In [26]:
from bs4 import BeautifulSoup
from datetime import datetime
from dateutil import parser
from IPython.core.display import display, HTML

import feedparser
import sqlite3
import pandas as pd
import pytz

/var/folders/fz/x86178z11qzd06x6yk5g9tz40000gn/T/ipykernel_62383/2701006436.py:5: DeprecationWarning: Importing display from IPython.core.display is deprecated since IPython 7.14, please import from IPython display
  from IPython.core.display import display, HTML


In [27]:
# Clean HTML tags from content
def clean_html(content):
    soup = BeautifulSoup(content, "html.parser")
    return soup.get_text()

# Function to fetch Google Alert RSS feed
def fetch_google_alerts_rss(rss_url):
    feed = feedparser.parse(rss_url)
    alert_data = []
    
    for entry in feed.entries:
        title = entry.title
        link = entry.link
        content = entry.content[0].value if 'content' in entry else entry.summary
        cleaned_content = clean_html(content)

        # Get publication date
        published_date = entry.published if 'published' in entry else None

        alert_data.append({
            "title": title,
            "link": link,
            "content": cleaned_content,
            "published_date": published_date  # Add publication date
        })
    
    return alert_data

# Function to save content into SQLite database
def save_to_database(data):
    conn = sqlite3.connect('google_alerts_rss.db')
    c = conn.cursor()

    # Create database table, including the published_date field
    c.execute('''CREATE TABLE IF NOT EXISTS alerts
                 (title TEXT, link TEXT UNIQUE, content TEXT, published_date TEXT)''')

    for item in data:
        try:
            c.execute("INSERT INTO alerts (title, link, content, published_date) VALUES (?, ?, ?, ?)",
                      (item["title"], item["link"], item["content"], item["published_date"]))
        except sqlite3.IntegrityError:
            pass  # Skip duplicate data

    conn.commit()
    conn.close()

# Function to convert database content to pandas dataframe
def convert_to_dataframe():
    conn = sqlite3.connect('google_alerts_rss.db')
    df = pd.read_sql_query("SELECT * FROM alerts ORDER BY published_date DESC", conn)  # Sort by publication date
    conn.close()
    
    # Convert link column to HTML anchor tag
    df['link'] = df['link'].apply(lambda x: f'<a href="{x}" target="_blank">{x}</a>')
    
    return df



In [28]:

# List of RSS URLs
rss_urls = [
    "https://www.google.com/alerts/feeds/03194265231041718071/2961366103475912817",  # Apple
    "https://www.google.com/alerts/feeds/03194265231041718071/17958653370686036725",  # Microsoft
    "https://www.google.com/alerts/feeds/03194265231041718071/12430836148975140862",  # Nasdaq

    "https://www.google.com/alerts/feeds/03194265231041718071/12043358214469963462",  # Clean energy（Industry
]

# Fetch and save data
for rss_url in rss_urls:
    alert_data = fetch_google_alerts_rss(rss_url)
    save_to_database(alert_data)

# Convert to DataFrame
df = convert_to_dataframe()

# Display styled HTML table
html_table = df.head(50).to_html(escape=False)
html_table_with_styles = f"""
    <style>
        table {{
            width: 100%;
            border-collapse: collapse;
        }}
        th, td {{
            border: 1px solid black;
            padding: 8px;
            text-align: left;
        }}
        td {{
            word-wrap: break-word;
            max-width: 200px;
            overflow: hidden;
            text-overflow: ellipsis;
        }}
    </style>
    {html_table}
"""
display(HTML(html_table_with_styles))

,title,link,content,published_date
0,"Evolus Reports Inducement Grants Under Nasdaq Listing Rule 5635(c)(4) - February 7, 2025",https://www.google.com/url?rct=j&sa=t&url=https://www.biospace.com/press-releases/evolus-reports-inducement-grants-under-nasdaq-listing-rule-5635c4-february-7-2025&ct=ga&cd=CAIyHDA1ODJiMTI2ZWI4ODdmYmE6Y28udWs6ZW46R0I&usg=AOvVaw1p9ZcBuLl1-V7SGMGqgNBl,"(NASDAQ: EOLS), a performance beauty company with a focus on building an aesthetic portfolio of consumer brands, today reported the grant of non- ...",2025-02-10T03:43:15Z
1,Bill Gates reveals he worked 80-hour weeks coding software while fearing Microsoft's downfall,https://www.google.com/url?rct=j&sa=t&url=https://www.hindustantimes.com/trending/bill-gates-reveals-he-worked-80-hour-weeks-coding-software-while-fearing-microsoft-s-downfall-101739155327163.html&ct=ga&cd=CAIyGmMzNmE5YzgyNTI1MTRkYmU6Y29tOmVuOlVT&usg=AOvVaw2ubLZG91H74oPFEq8Eq-Ee,"Bill Gates revealed he worked 80-hour weeks at Microsoft for years, fearing failure even after becoming a billionaire. | Trending.",2025-02-10T03:04:45Z
2,JA Solar's DeepBlue 4.0 Pro Energizes Shams Power's PV Project at Coca-Cola Faisalabad,https://www.google.com/url?rct=j&sa=t&url=https://www.morningstar.com/news/pr-newswire/20250209cn14221/ja-solars-deepblue-40-pro-energizes-shams-powers-pv-project-at-coca-cola-faisalabad&ct=ga&cd=CAIyGjdhYzg5NGQ3NTQwNjQ3YWE6Y29tOmVuOlVT&usg=AOvVaw2d0QrZzWUuNFrtIPfdS-z9,... green manufacturing efforts and aligns with Pakistan's growing demand for clean energy. ... Power with advancing renewable energy adoption across ...,2025-02-10T03:04:00Z
3,"G42, Microsoft unveil Responsible AI Foundation - Gulf Business",https://www.google.com/url?rct=j&sa=t&url=https://gulfbusiness.com/g42-microsoft-unveil-responsible-ai-foundation/&ct=ga&cd=CAIyGmMzNmE5YzgyNTI1MTRkYmU6Y29tOmVuOlVT&usg=AOvVaw2nTcfJpP3qQJCGR1GSGfFy,"G42 and Microsoft have officially launched the Responsible AI Foundation, marking the first initiative of its kind in the Middle East.",2025-02-10T03:01:35Z
4,Kendrick Lamar's full Apple Music Halftime Performance - Yahoo Sports,https://www.google.com/url?rct=j&sa=t&url=https://sports.yahoo.com/kendrick-lamars-full-apple-music-021108450.html&ct=ga&cd=CAIyHDYwYmI2ZjUxN2Q3ODYxZGI6Y28udWs6ZW46R0I&usg=AOvVaw0ms8PZAIQl3aPZHnXdDQRf,Watch Kendrick Lamar's full performance from the Apple Music Super Bowl LIX Halftime Show.,2025-02-10T02:56:55Z
5,"Big Apple launch this week? From new iPhone SE, MacBook Air, iPads; here's what to expect",https://www.google.com/url?rct=j&sa=t&url=https://www.businesstoday.in/technology/news/story/big-apple-launch-this-week-from-new-iphone-se-macbook-air-ipads-heres-what-to-expect-463965-2025-02-10&ct=ga&cd=CAIyHDYwYmI2ZjUxN2Q3ODYxZGI6Y28udWs6ZW46R0I&usg=AOvVaw0uqzOntRcBElmL_x1WfsTm,"Apple is reportedly preparing for a surprise product launch this week, potentially unveiling a new iPhone SE, updated iPads, and a MacBook Air.",2025-02-10T02:49:51Z
6,Capital Clean Energy Carriers (CCEC) Gets a Buy from Stifel Nicolaus | Markets Insider,https://www.google.com/url?rct=j&sa=t&url=https://markets.businessinsider.com/news/stocks/capital-clean-energy-carriers-ccec-gets-a-buy-from-stifel-nicolaus-1034331143&ct=ga&cd=CAIyGjdhYzg5NGQ3NTQwNjQ3YWE6Y29tOmVuOlVT&usg=AOvVaw1WJ_6ZLOOPCiIY8bb1jPqo,"In a report released on February 7, Benjamin Nolan from Stifel Nicolaus maintained a Buy rating on Capital Clean Energy Carriers (CCEC – Researc.",2025-02-10T02:32:47Z
7,Europe Phases Out Russian Fossil Fuels | Transitioning to a Secure and Sustainable Energy Future,https://www.google.com/url?rct=j&sa=t&url=https://www.youtube.com/watch%3Fv%3DL4tYzWh4JF8&ct=ga&cd=CAIyGjdhYzg5NGQ3NTQwNjQ3YWE6Y29tOmVuOlVT&usg=AOvVaw2_pOu906PXLaGJH6kMQ6Jh,... renewable sources | energy supply diversification | sustainable EU future | European energy crisis | Russia energy dependence | clean energy,2025-02-10T02:32:04Z
8,"Apple Invites is a great idea, but Google Photos and Calendar could make it even be